<a href="https://colab.research.google.com/github/sania96/Gen-AI/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [13]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sania963","key":"f499209cc02d78caa7b50724ad17e55e"}'}

In [14]:
! cp kaggle.json ~/.kaggle/

In [15]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [16]:
!kaggle datasets download -d odins0n/ucf-crime-dataset

100% 11.0G/11.0G [09:34<00:00, 22.9MB/s]
100% 11.0G/11.0G [09:34<00:00, 20.6MB/s]


In [17]:
import zipfile

with zipfile.ZipFile("/content/ucf-crime-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/ucf-crime-dataset/")


In [18]:
# Delete the zip file to free up disk space
import os
os.remove("/content/ucf-crime-dataset.zip")

In [19]:
import os

# Path to the directory containing the unzipped files
directory_path = "/content/ucf-crime-dataset/"

# List the contents of the directory
contents = os.listdir(directory_path)

# Print the list of contents
print(contents)

['Train', 'Test']


In [21]:
!pip install torch torchvision Pillow numpy tqdm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00


In [28]:
import os
from multiprocessing import Pool
import torch
import argparse
import numpy as np
from tqdm import tqdm
from PIL import Image
from torch.autograd import Variable
from i3dpt import I3D
import math


In [29]:
import os
from multiprocessing import Pool
import argparse
import numpy as np
from tqdm import tqdm
from i3dpt import I3D  # Assuming you have i3dpt.py in your working directory

# Function to load RGB images
def load_rgb_images(folder_path, num_images=50):
    images = []
    for root, dirs, files in os.walk(folder_path):
        image_count = 0
        for file in sorted(files):
            if file.endswith('.png') and image_count < num_images:
                image_path = os.path.join(root, file)
                images.append(image_path)
                image_count += 1
    return images

# Function to extract features for a single video
def extract_features(video_dir, output_dir, load_model='model_rgb.pth', batch_size=20, frequency=16):
    # Setup the model
    i3d = I3D(400, modality='rgb', dropout_prob=0, name='inception')
    i3d.eval()
    i3d.load_state_dict(torch.load(load_model))
    i3d.cuda()

    # Load RGB images
    image_paths = load_rgb_images(video_dir)

    # Extract features
    features = []
    for start_index in tqdm(range(0, len(image_paths), batch_size)):
        batch_paths = image_paths[start_index:start_index+batch_size]
        batch_data = [load_frame(image_path) for image_path in batch_paths]
        batch_features = forward_batch(batch_data, i3d)
        features.append(batch_features)

    # Save features
    features = np.concatenate(features, axis=0)
    video_name = os.path.basename(video_dir)
    save_file = os.path.join(output_dir, f'{video_name}_i3d.npy')
    np.save(save_file, features)
    print(f'Features extracted and saved for {video_name}')

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--input_dir', default='Train', type=str, help='Path to input directory containing subfolders')
    parser.add_argument('--output_dir', default='UCF_ten', type=str, help='Path to output directory')
    parser.add_argument('--num_images_per_folder', default=50, type=int, help='Number of images to process per subfolder')
    parser.add_argument('--load_model', default='model_rgb.pth', type=str, help='Path to pre-trained model')
    parser.add_argument('--batch_size', default=20, type=int, help='Batch size for feature extraction')
    parser.add_argument('--frequency', default=16, type=int, help='Frequency for feature extraction')
    args = parser.parse_args()

    # Create output directory if it doesn't exist
    os.makedirs(args.output_dir, exist_ok=True)

    # Iterate over subfolders in the input directory
    for class_folder in os.listdir(args.input_dir):
        class_folder_path = os.path.join(args.input_dir, class_folder)
        if os.path.isdir(class_folder_path):
            print(f'Processing folder: {class_folder}')
            for video_folder in os.listdir(class_folder_path):
                video_folder_path = os.path.join(class_folder_path, video_folder)
                if os.path.isdir(video_folder_path):
                    extract_features(video_folder_path, args.output_dir, args.load_model, args.batch_size, args.frequency)


usage: colab_kernel_launcher.py [-h] [--input_dir INPUT_DIR] [--output_dir OUTPUT_DIR]
                                [--num_images_per_folder NUM_IMAGES_PER_FOLDER]
                                [--load_model LOAD_MODEL] [--batch_size BATCH_SIZE]
                                [--frequency FREQUENCY]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-16178d66-30b6-4481-9ca0-634fd027f426.json


SystemExit: 2

usage: colab_kernel_launcher.py [-h] [--mode MODE] [--load_model LOAD_MODEL]
                                [--input_dir INPUT_DIR] [--output_dir OUTPUT_DIR]
                                [--batch_size BATCH_SIZE] [--sample_mode SAMPLE_MODE]
                                [--frequency FREQUENCY]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-16178d66-30b6-4481-9ca0-634fd027f426.json


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
